In [1]:
import os
import io
import tweepy
from google.cloud import bigquery
from google.oauth2 import service_account

### Connect to GBQ

In [2]:
# Do our imports for the code
from google.cloud import bigquery
from google.oauth2 import service_account

# Set authentication info
service_path = "/Users/natebender/Desktop/repo/footprints/footprints_audience/"
service_file = 'footprintsrunning-d15e651db9a6.json' # change this to your authentication information  
gbq_proj_id = 'footprintsrunning' # change this to your poroject. 

# And this should stay the same. 
private_key = service_path + service_file

# Now we pass in our credentials so that Python has permission to access our project.
credentials = service_account.Credentials.from_service_account_file(service_path + service_file)

# And finally we establish our connection
gbqclient = bigquery.Client(credentials = credentials, project=gbq_proj_id)

In [3]:
# Test connection
clean_files = "/Users/natebender/Desktop/repo/footprints/footprints_audience/data"
os.listdir(clean_files)

['20220906_000000_trailrunningtweets.csv',
 '20220216_000000_trailrunningtweets.csv',
 '20220105_0112_000000_trailrunningtweets.csv',
 '20220104_000000_trailrunningtweets.csv',
 '1228-1231_000000_trailrunningtweets.csv',
 '20220131_000000_trailrunningtweets.csv',
 '20220116_0123_000000_trailrunningtweets.csv']

### Connect to Tweepy

In [4]:
# Set connection to Tweepy

# From the "project set-up" experience
api_key = 'e9hKR9ngqwXalms6q4IndwoAz'
api_key_secret = 'mQzPQWyQvO3ikOqMGjJfKpbFTMcSJKNE9vvywpuSi1Jv0HAjGt'
bearer_token = 'AAAAAAAAAAAAAAAAAAAAADMxgwEAAAAAcEBEj8u5Qy'

In [5]:
tweepyclient = tweepy.Client(bearer_token, wait_on_rate_limit=True)

In [ ]:
# You should have data in the file that you can see via the console. Let's test it here.
#query = (
#    "SELECT * "
#    "FROM `footprintsrunning.twitter_scraping.trail_running`" # change to *your*` project ID, data set ID, and table name! 
#)

#results = gbqclient.query(query)

#print("Let's print it ugly.")
#print(list(results))

### Clean data

In [6]:
import pandas as pd
import re
import glob
import nltk
from nltk import word_tokenize, sent_tokenize, FreqDist
from nltk.stem import LancasterStemmer, WordNetLemmatizer
from nltk.tokenize import TweetTokenizer
from nltk.corpus import stopwords
import preprocessor as p
import numpy as np
words = set(nltk.corpus.words.words())
sw=set(stopwords.words('english'))

In [7]:
# Upload all local csv files to single pandas dataframe

path = 'data/' # use your path

all_files = glob.glob(os.path.join(path, "*.csv"))

tweets = pd.concat((pd.read_csv(f) for f in all_files), ignore_index=True)

In [8]:
def clean_tweet(tweet):
    if type(tweet) == float:
        return ""
    temp = tweet.lower()
    temp = re.sub("'", "", temp) # to avoid removing contractions in english
    temp = re.sub("@[A-Za-z0-9_]+","", temp)  # remove mentions
    temp = re.sub("#[A-Za-z0-9_]+","", temp)  # remove hashtags
    temp = re.sub(r'http\S+', '', temp)  # remove links
    temp = re.sub('[()!?]', ' ', temp)  # remove punct
    temp = re.sub('\[.*?\]',' ', temp)  # remove punct
    temp = re.sub("[^a-z0-9]"," ", temp) # remove non-alphanumeric
    temp = temp.split()  # tokenize
    temp = [w for w in temp if w not in sw]  # remove stopwords
    temp = " ".join(word for word in temp)  # rejoin into tweets again
    return temp

In [9]:
tweets['fulltext_clean'] = tweets['full_text'].apply(clean_tweet)  # create new column of cleaned tweets

In [10]:
tweets_final = tweets.drop('full_text', axis=1)  # drop orig full_text column

In [11]:
tweets['desc_clean'] = tweets['description'].apply(clean_tweet)  # create new column of cleaned tweets

In [12]:
tweets_final = tweets.drop('description', axis=1)  # drop orig full_text column

In [13]:
tweets_final.head()

,user_id,screen_name,location,friends_count,followers_count,totaltweets,date_created,tweet_id,retweetcount,full_text,fulltext_clean,desc_clean
0,80147238,mendonca1969,Canada,675,594,35759,2022-09-04 23:58:16+00:00,1566576408469639171,0,VINYL NOW PLAYING: Jessie Ware “Devotion 10 An...,vinyl playing jessie ware devotion 10 annivers...,advocate environment animals lgbtqi lover viny...
1,1303924252090028032,RoseGarde4,NaN,103,46,1081,2022-09-04 23:56:51+00:00,1566576052578783232,0,A glimpse of today's adventure with my husband...,glimpse todays adventure husband,canadian encourage write create art garden exe...
2,1566571989564809217,RjpsTop15,NaN,9,1,9,2022-09-04 23:56:47+00:00,1566576038376869889,0,VINYL NOW PLAYING: Jessie Ware “Devotion 10 An...,vinyl playing jessie ware devotion 10 annivers...,
3,1506286335623061516,OrvilleBernard6,NaN,297,78,22,2022-09-04 23:55:48+00:00,1566575790170701824,1,Running is Big ocean 🌊#MoonFit #GameFi #Dotsa...,running big ocean,coming soon
4,1512801104135962628,lean_parenthood,NaN,3350,1367,2087,2022-09-04 23:50:49+00:00,1566574535717036034,0,"3 mile #run done tonight. Wet, humid night, bu...",3 mile done tonight wet humid night get,documenting journey getting lean new dad weigh...


### Upload data

In [14]:
import pandas_gbq

In [15]:
table_id = 'footprintsrunning.twitter_scraping.trail_running'

### Note for JC: Need to update so table is not replaced each time, but instead code checks for duplicate tweet IDs and skips if ID is already in GBQ. Some combination of first checking for dupes and then if_exists="append" ??

In [16]:
pandas_gbq.to_gbq(tweets_final, table_id, gbq_proj_id, if_exists="replace")

100%|███████████████████████████████████████████| 1/1 [00:00<00:00, 2432.89it/s]


In [17]:
table = gbqclient.get_table(table_id)  # Make an API request.

# View table properties
print(
    "Got table '{}.{}.{}'.".format(table.project, table.dataset_id, table.table_id)
)
print("Table schema: {}".format(table.schema))
print("Table description: {}".format(table.description))
print("Table has {} rows".format(table.num_rows))

Got table 'footprintsrunning.twitter_scraping.trail_running'.
Table schema: [SchemaField('user_id', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('screen_name', 'STRING', 'NULLABLE', None, (), None), SchemaField('location', 'STRING', 'NULLABLE', None, (), None), SchemaField('friends_count', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('followers_count', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('totaltweets', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('date_created', 'STRING', 'NULLABLE', None, (), None), SchemaField('tweet_id', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('retweetcount', 'INTEGER', 'NULLABLE', None, (), None), SchemaField('full_text', 'STRING', 'NULLABLE', None, (), None), SchemaField('fulltext_clean', 'STRING', 'NULLABLE', None, (), None), SchemaField('desc_clean', 'STRING', 'NULLABLE', None, (), None)]
Table description: None
Table has 89354 rows
